# 🚀 Proyecto ELT con SpaceX API  
## Notebook 01 – Extracción y Carga

En este notebook se realiza la **extracción de datos desde la API pública de SpaceX** y el **almacenamiento en Delta Lake**.  

Se siguen los pasos de la consigna:  
1. Extracción de **2 o más endpoints**.  
2. Uso de al menos un **endpoint dinámico (actualizable)** y otro **estático**.  
3. Guardado en **formato Delta Lake**.  
4. Aplicación de **extracción incremental y full** según corresponda.  


In [2]:
# =========================
# CELDA DE CONFIGURACIÓN INICIAL
# =========================

import sys
from pathlib import Path
import logging

# --- RUTA DEL PROYECTO ---
# Detecta automáticamente la raíz del proyecto buscando la carpeta "src"
def find_project_root(marker="src"):
    path = Path().cwd()
    for _ in range(5):  # sube hasta 5 niveles si es necesario
        if (path / marker).exists():
            return path
        path = path.parent
    raise FileNotFoundError(f"No se encontró la carpeta '{marker}' en los niveles superiores.")

project_root = find_project_root()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print(f"Project root detectado en: {project_root}")

# --- IMPORTS DEL PROYECTO ---
from src.extract import fetch_data
from src.load import save_to_delta
from src.utils import setup_logger

# --- LOGGER ---
logger = setup_logger()
logger.info("Inicio de ejecución del notebook")


Project root detectado en: c:\Users\MONSO\OneDrive\Escritorio\Final-DataEngineering


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

## 🔗 Endpoints seleccionados

- **Dinámicos:**  
  - `launches/latest` → Último lanzamiento.  
  - `launches/upcoming` → Próximos lanzamientos.  

- **Estáticos:**  
  - `rockets` → Información de cohetes.  
  - `dragons` → Información de cápsulas Dragon.  

In [ ]:
# Extracción de datos
df_latest = fetch_data("latest_launch")
df_upcoming = fetch_data("upcoming_launches")
df_rockets = fetch_data("rockets")
df_dragons = fetch_data("dragons")

logger.info(f"Último lanzamiento: {len(df_latest)} registros")
logger.info(f"Próximos lanzamientos: {len(df_upcoming)} registros")
logger.info(f"Cohetes: {len(df_rockets)} registros")
logger.info(f"Dragons: {len(df_dragons)} registros")

## 💾 Guardado en Delta Lake

Se guarda cada DataFrame en formato **Delta Lake**:  
- Los **endpoints dinámicos** (`latest_launch`, `upcoming_launches`) se almacenan con **particiones por fecha (extracción incremental)**.  
- Los **endpoints estáticos** (`rockets`, `dragons`) se guardan en una única ruta (extracción full).  

In [ ]:
# Guardado en Delta Lake
save_to_delta(df_latest, "latest_launch", incremental=True)
save_to_delta(df_upcoming, "upcoming_launches", incremental=True)
save_to_delta(df_rockets, "rockets", incremental=False)
save_to_delta(df_dragons, "dragons", incremental=False)

logger.info("Todos los datasets fueron guardados correctamente en Delta Lake.")
